# Metrics Fetcher

Data fetched using the metrics fetcher application is stored in an SQLite database. This notebook will help analyze the content of this database.


Firstly, we will import dependencies and set constants for this analysis:


In [180]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf


db_path = "storage.db"
conn = sqlite3.connect(db_path)

Next would be to extract data from the database:

In [181]:
# region SQL queries to extract data
cpuMetrics = pd.read_sql_query("SELECT * FROM CPUMetricProvider WHERE 1=1 ORDER BY id", conn)
processMetrics = pd.read_sql_query("SELECT * FROM ProcessMetricProvider WHERE 1=1 ORDER BY id", conn)
memoryMetrics = pd.read_sql_query("SELECT * FROM RAMMetricProvider WHERE 1=1 ORDER BY id", conn)
storageMetrics = pd.read_sql_query("SELECT * FROM StorageMetricProvider WHERE 1=1 ORDER BY id", conn)
# networkMetrics = pd.read_sql_query("SELECT * FROM NetworkMetricProvider WHERE 1=1 ORDER BY id", conn)
scriptMetrics = pd.read_sql_query("SELECT * FROM ScriptData WHERE 1=1 ORDER BY id", conn)
# endregion

cpuMetrics.drop(columns="name", inplace=True)
cpuMetrics['timestamp'] = (cpuMetrics['timestamp'] // 100) * 100
cpuMetrics['counterU'] = cpuMetrics['timestamp'].astype(str) + cpuMetrics['counter'].astype(str)
cpuMetrics['counterU'] = cpuMetrics['counterU'].astype("int64")
# cpuMetrics = cpuMetrics.groupby(['counterU'], as_index=False).mean()
cpuMetrics.head()

,id,counter,usage,instructionsRetired,cycles,floatingPointOperations,temperature,timestamp,counterU
0,1,0,30.97320,0.0,0.0,0.0,0.0,1704935000,17049350000
1,2,1,5.12935,0.0,0.0,0.0,0.0,1704935000,17049350001
2,3,2,4.46969,0.0,0.0,0.0,0.0,1704935000,17049350002
3,4,3,5.77369,0.0,0.0,0.0,0.0,1704935000,17049350003
4,5,4,24.02100,0.0,0.0,0.0,0.0,1704935000,17049350004


In [182]:
processMetrics.drop(columns="name", inplace=True)
processMetrics['timestamp'] = (processMetrics['timestamp'] // 100) * 100
processMetrics['counterU'] = processMetrics['timestamp'].astype(str) + processMetrics['counter'].astype(str)
processMetrics['counterU'] = processMetrics['counterU'].astype("int64")
processMetrics.head()

,id,counter,processCount,activeProcess,activeWindow,bytesReadPerSecond,bytesWrittenPerSecond,timestamp,counterU
0,1,0,236.0,WindowsTerminal.exe,ICE,533873.0,2431480.0,1704935000,17049350000
1,2,2,235.0,WindowsTerminal.exe,ICE,425455.0,185232.0,1704935000,17049350002
2,3,3,235.0,WindowsTerminal.exe,ICE,1143460.0,150020.0,1704935000,17049350003
3,4,4,234.0,WindowsTerminal.exe,ICE,157642.0,216894.0,1704935000,17049350004
4,5,5,237.0,chrome.exe,My Drive - Google Drive - Google Chrome,2422430.0,2428540.0,1704935000,17049350005


In [183]:
memoryMetrics.drop(columns="name", inplace=True)
memoryMetrics['timestamp'] = (memoryMetrics['timestamp'] // 100) * 100
memoryMetrics['counterU'] = memoryMetrics['timestamp'].astype(str) + memoryMetrics['counter'].astype(str)
memoryMetrics['counterU'] = memoryMetrics['counterU'].astype("int64")
memoryMetrics = memoryMetrics.groupby(['counter'], as_index=False).mean()
memoryMetrics.head()

,counter,id,available,committed,pageFaults,timestamp,counterU
0,1,1.5,1.042870e+10,6.614860e+09,20839.445,1.704935e+09,1.704935e+10
1,2,3.0,1.036550e+10,6.619230e+09,2705.250,1.704935e+09,1.704935e+10
2,3,4.0,1.043950e+10,6.535710e+09,2359.370,1.704935e+09,1.704935e+10
3,5,5.0,1.016410e+10,6.902400e+09,28631.700,1.704935e+09,1.704935e+10
4,6,6.0,9.644950e+09,7.780360e+09,58525.700,1.704935e+09,1.704935e+10


In [184]:
storageMetrics.drop(columns="name", inplace=True)
storageMetrics['timestamp'] = (storageMetrics['timestamp'] // 100) * 100
storageMetrics['counterU'] = storageMetrics['timestamp'].astype(str) + storageMetrics['counter'].astype(str)
storageMetrics['counterU'] = storageMetrics['counterU'].astype('Int64')
storageMetrics = storageMetrics.groupby(['counter'], as_index=False).mean()
storageMetrics.head()

,counter,id,read,write,transferRate,timestamp,counterU
0,1,1.0,464097.00,14108600.0,14572600.0,1.704935e+09,17049350001.0
1,2,2.0,9395.95,580609.0,590005.0,1.704935e+09,17049350002.0
2,3,3.0,0.00,142101.0,142101.0,1.704935e+09,17049350003.0
3,4,4.5,617607.60,1239224.0,1856832.0,1.704935e+09,17049350004.0
4,6,6.5,755052.45,1714380.0,2469430.0,1.704935e+09,17049351006.0


In [185]:
scriptMetrics['timestamp'] = (scriptMetrics['timestamp'] // 100) * 100
scriptMetrics['counterU'] = scriptMetrics['timestamp'].astype(str) + scriptMetrics['counter'].astype(str)
scriptMetrics = scriptMetrics.groupby(['key', 'counterU'], as_index=False).mean()
scriptMetrics['counterU'] = scriptMetrics['counterU'].astype('Int64')
scriptMetrics.head()

,key,counterU,id,counter,value,timestamp
0,ProcessorPerformance,1704938300329,1.0,329.0,115.0510,1.704938e+09
1,ProcessorPerformance,1704938300330,2.0,330.0,123.5160,1.704938e+09
2,ProcessorPerformance,1704938300331,3.0,331.0,90.8270,1.704938e+09
3,ProcessorPerformance,1704938300332,4.0,332.0,121.9960,1.704938e+09
4,ProcessorPerformance,1704938300333,5.0,333.0,75.5344,1.704938e+09


In [186]:
scriptMetrics = scriptMetrics.pivot(index='counterU', columns='key', values='value').reset_index()
scriptMetrics.head()

key,counterU,ProcessorPerformance
0,1704938300329,115.0510
1,1704938300330,123.5160
2,1704938300331,90.8270
3,1704938300332,121.9960
4,1704938300333,75.5344


Now that we have all the tables we require, we will create labels for them.
The final goal is to create one single table with all the features we have seen,
so we will use a common prefix for the labeling.

In [187]:
# cpuMetrics, processMetrics, memoryMetrics, storageMetrics, scriptMetrics

# This indicates whether for each reading the CPU is in good state or not.
# We will use "1" and "0" to represent bad state and good state in labels.
cpuMetrics["t_cpu_label"] = 0
cpuMetrics["t_cpu_label"] = np.where((cpuMetrics["usage"] > 90), 1, cpuMetrics["t_cpu_label"]);

processMetrics["t_proc_label"] = 0;
processMetrics["t_proc_label"] = np.where((processMetrics["bytesReadPerSecond"] < 5000000), 1, processMetrics["t_proc_label"]);

memoryMetrics["t_mem_label"] = 0;
memoryMetrics["t_mem_label"] = np.where((memoryMetrics["pageFaults"] > 500000), 1, memoryMetrics["t_mem_label"]);

storageMetrics["t_store_label"] = 0;
storageMetrics["t_store_label"] = np.where((storageMetrics["transferRate"] < 9999999), 1, storageMetrics["t_store_label"]);

scriptMetrics["t_script_label"] = 0;
scriptMetrics["t_script_label"] = np.where((scriptMetrics["ProcessorPerformance"] < 9999999), 1, scriptMetrics["t_script_label"]);

# Merge the tables
cpuMetrics.drop(columns=["counter", "id", "timestamp"], inplace=True)
processMetrics.drop(columns=["counter", "id", "timestamp"], inplace=True)
memoryMetrics.drop(columns=["counter", "id", "timestamp"], inplace=True)
storageMetrics.drop(columns=[ "counter", "id", "timestamp"], inplace=True)

combined_data = pd.merge(cpuMetrics, processMetrics, on='counterU', how='inner')
combined_data = pd.merge(combined_data, memoryMetrics, on='counterU', how='inner')
combined_data = pd.merge(combined_data, storageMetrics, on='counterU', how='inner')
combined_data = pd.merge(combined_data, scriptMetrics, on='counterU', how='inner')
combined_data = combined_data.dropna()
combined_data.head()

,usage,instructionsRetired,cycles,floatingPointOperations,temperature,counterU,t_cpu_label,processCount,activeProcess,activeWindow,...,available,committed,pageFaults,t_mem_label,read,write,transferRate,t_store_label,ProcessorPerformance,t_script_label
0,9.58457,0.0,0.0,0.0,0.0,1704938300332,0,260.0,chrome.exe,Metrics Fetcher - Google Chrome,...,9.455550e+09,8.379360e+09,1108.5000,0,11830.50,309633.0,321463.0,1,121.996,1
1,9.58457,0.0,0.0,0.0,0.0,1704938300332,0,260.0,chrome.exe,Metrics Fetcher - Google Chrome,...,9.455550e+09,8.379360e+09,1108.5000,0,11830.50,309633.0,321463.0,1,121.996,1
2,2.36571,0.0,0.0,0.0,0.0,1704938300334,0,260.0,chrome.exe,Metrics Fetcher - Google Chrome,...,9.480595e+09,8.341630e+09,1230.8855,0,16766.50,165211.0,181978.0,1,116.932,1
3,3.27768,0.0,0.0,0.0,0.0,1704938400337,0,260.0,chrome.exe,Winter Lofi ❄ Lofi Keep You Safe 🍂 Calm Your M...,...,9.456330e+09,8.355590e+09,2623.2250,0,1641.57,111627.0,113268.0,1,112.802,1
4,1.88729,0.0,0.0,0.0,0.0,1704938400339,0,260.0,chrome.exe,Winter Lofi ❄ Lofi Keep You Safe 🍂 Calm Your M...,...,9.473650e+09,8.338820e+09,469.1750,0,10230.70,104762.0,114993.0,1,66.594,1


Now that we have our combined table, we can proceed with the steps to create a model. But we need to encode the string values we have.
One-hot encoder from `Pandas` is preferred over a `LabelEncoder` because the data strings identified are nominal.

In [188]:
columns_to_encode = ['activeProcess', 'activeWindow']
for column in columns_to_encode:
    combined_data[column] = combined_data[column].str.strip().replace(to_replace=r"\s", value='_', regex=True)
combined_data_encoded = pd.get_dummies(combined_data, columns=columns_to_encode, prefix='one_hot', drop_first=True, dummy_na=False, dtype=int)
generated_columns = set(combined_data_encoded.columns) - set(combined_data.columns)
combined_data = combined_data_encoded
combined_data.fillna(0, inplace=True)
combined_data.drop(['counterU'], axis=1, inplace=True)
combined_data.head()

,usage,instructionsRetired,cycles,floatingPointOperations,temperature,t_cpu_label,processCount,bytesReadPerSecond,bytesWrittenPerSecond,t_proc_label,...,one_hot_pdhprocessorperformance_-_Google_Search_-_Google_Chrome,one_hot_settings.ts_-_ICE_-_Visual_Studio_Code,one_hot_stress.txt_-_Notepad,one_hot_telephony-plugin-selector.ts_-_SCVConnectorApp_-_Visual_Studio_Code,one_hot_uplandsoftware_/_ice_/_Pull_Request_#3714:_ICE-12712:_Create_dev_package_that_contains_ACW_Wrapper_changes_—_Bitbucket_-_Google_Chro,one_hot_uplandsoftware_/_ice_/_Pull_Request_#3718:_ICE-12635_SCV_-_VCRs_for_internal_inbound_calls_report_as_transfers_—_Bitbucket_-_Google_Chro,one_hot_uplandsoftware_/_ice_/_Pull_Request_#3725:_ICE-10908:_Build_ServiceNow_Tunnel_Solution_(Mitigation_PR_13)_(Bootstrap_Error_Handler_Decorator)_—_Bitbucket_-_Google_Chro,one_hot_windows_check_cpu_temp_-_Google_Search_-_Google_Chrome,one_hot_windows_check_cpu_temp_c++pdh_-_Google_Search_-_Google_Chrome,one_hot_❄️_Chillhop_Essentials_•_Winter_2023_[cozy_instrumentals_/_chill_lofi_beats]_-_YouTube_-_Google
0,9.58457,0.0,0.0,0.0,0.0,0,260.0,1256040.0,291855.0,1,...,0,0,0,0,0,0,0,0,0,0
1,9.58457,0.0,0.0,0.0,0.0,0,260.0,139821.0,115643.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2.36571,0.0,0.0,0.0,0.0,0,260.0,179618.0,75496.1,1,...,0,0,0,0,0,0,0,0,0,0
3,3.27768,0.0,0.0,0.0,0.0,0,260.0,1315710.0,149866.0,1,...,0,0,0,0,0,0,0,0,0,0
4,1.88729,0.0,0.0,0.0,0.0,0,260.0,63762.1,92716.4,1,...,0,0,0,0,0,0,0,0,0,0


In [189]:
generated_columns

{'one_hot_73%_complete',
 'one_hot_7zG.exe',
 'one_hot_Add_Page_-_How_to_Setup_Genesys_Engage_as_a_Developer_-_InGenius_Product_Development_-_Confluence_-_Google_Chrome',
 'one_hot_ApplicationFrameHost.exe',
 'one_hot_Asphalt_9:_Legends',
 'one_hot_Asphalt_9:_Legends_|_Xbox_-_Google_Chrome',
 'one_hot_Benchmark',
 'one_hot_Benchmarks_Downloads_|_TechSpot_-_Google_Chrome',
 'one_hot_Break_free_from_your_passwords_-_Google_Chrome',
 'one_hot_Chat_|_Microsoft_Teams_classic',
 'one_hot_Code.exe',
 'one_hot_Collecting_Performance_Data_-_Win32_apps_|_Microsoft_Learn_-_Google_Chrome',
 'one_hot_Confirm_your_account_recovery_settings_-_Google_Chrome',
 'one_hot_Counters_Monitoring_CPU_Usage_|_Network_Wrangler_–_Tech_Blog_-_Google_Chro',
 'one_hot_Create_branch_—_Bitbucket_-_Google_Chro',
 'one_hot_Crossover.exe',
 'one_hot_CrystalDiskMark_-_Crystal_Dew_World_[en]_-_Google_Chrome',
 'one_hot_CrystalDiskMark_8.0.4_x64_[Admin]',
 'one_hot_DB_Browser_for_SQLite.exe',
 'one_hot_DB_Browser_for_SQLit

In [190]:
label_columns = [col for col in combined_data.columns if '_label' in col.lower()]
feature_columns = [col for col in combined_data.columns if col not in label_columns]

num_timesteps = 1
num_features = len(feature_columns)

X = combined_data[feature_columns]
X = X.values.reshape((X.shape[0], num_timesteps, X.shape[1]))
y = combined_data[label_columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = tf.constant(X_train, dtype=tf.float32)
X_test = tf.constant(X_test, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.int32)
y_test = tf.constant(y_test, dtype=tf.int32)

In [191]:
X_train

<tf.Tensor: shape=(3274, 1, 172), dtype=float32, numpy=
array([[[ 6.55935,  0.     ,  0.     , ...,  0.     ,  0.     ,
          0.     ]],

       [[ 5.67742,  0.     ,  0.     , ...,  0.     ,  0.     ,
          0.     ]],

       [[11.5573 ,  0.     ,  0.     , ...,  0.     ,  0.     ,
          0.     ]],

       ...,

       [[ 5.04017,  0.     ,  0.     , ...,  0.     ,  0.     ,
          0.     ]],

       [[ 8.09318,  0.     ,  0.     , ...,  0.     ,  0.     ,
          0.     ]],

       [[ 8.59966,  0.     ,  0.     , ...,  0.     ,  0.     ,
          0.     ]]], dtype=float32)>

In [192]:
y_train

<tf.Tensor: shape=(3274, 5), dtype=int32, numpy=
array([[0, 1, 0, 1, 1],
       [0, 1, 0, 1, 1],
       [0, 1, 0, 1, 1],
       ...,
       [0, 1, 0, 1, 1],
       [0, 1, 0, 1, 1],
       [0, 1, 0, 1, 1]])>

In [193]:
# input_layer = tf.keras.layers.Input(shape=(num_timesteps, num_features))
input_layer = tf.keras.layers.Input(shape=(None, num_features))
lstm_layer = tf.keras.layers.LSTM(units=50, activation='tanh')(input_layer)

# Define separate output layers for each label
output_t_cpu = tf.keras.layers.Dense(1, activation='sigmoid', name='t_cpu_label')(lstm_layer)
output_t_proc = tf.keras.layers.Dense(1, activation='sigmoid', name='t_proc_label')(lstm_layer)
output_t_mem = tf.keras.layers.Dense(1, activation='sigmoid', name='t_mem_label')(lstm_layer)
output_t_store = tf.keras.layers.Dense(1, activation='sigmoid', name='t_store_label')(lstm_layer)
output_t_script = tf.keras.layers.Dense(1, activation='sigmoid', name='t_script_label')(lstm_layer)

model = tf.keras.models.Model(inputs=input_layer, outputs=[output_t_cpu, output_t_proc, output_t_mem, output_t_store, output_t_script])
# model.add(tf.keras.layers.Dense(32, activation='relu'))
# model.add(tf.keras.layers.Dense(16, activation='relu'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, None, 172)]          0         []                            
                                                                                                  
 lstm_10 (LSTM)              (None, 50)                   44600     ['input_11[0][0]']            
                                                                                                  
 t_cpu_label (Dense)         (None, 1)                    51        ['lstm_10[0][0]']             
                                                                                                  
 t_proc_label (Dense)        (None, 1)                    51        ['lstm_10[0][0]']             
                                                                                           

Now we train the model 🥷😍

In [194]:
%load_ext tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/fit/", histogram_freq=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

model.fit(X_train, [y_train[:, 0], y_train[:, 1], y_train[:, 2], y_train[:, 3], y_train[:, 4]], epochs=1000, batch_size=32, validation_split=0.2, callbacks=[early_stopping, tensorboard_callback])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/1000
82/82 [==============================] - 4s 10ms/step - loss: 3.4493 - t_cpu_label_loss: 1.0556 - t_proc_label_loss: 0.7649 - t_mem_label_loss: 0.4942 - t_store_label_loss: 0.4491 - t_script_label_loss: 0.6854 - t_cpu_label_accuracy: 0.0374 - t_proc_label_accuracy: 0.3574 - t_mem_label_accuracy: 0.9683 - t_store_label_accuracy: 0.9488 - t_script_label_accuracy: 0.5403 - val_loss: 2.5538 - val_t_cpu_label_loss: 0.7997 - val_t_proc_label_loss: 0.5741 - val_t_mem_label_loss: 0.3481 - val_t_store_label_loss: 0.3384 - val_t_script_label_loss: 0.4936 - val_t_cpu_label_accuracy: 0.0824 - val_t_proc_label_accuracy: 0.9573 - val_t_mem_label_accuracy: 1.0000 - val_t_store_label_accuracy: 0.9603 - val_t_script_label_accuracy: 1.0000
Epoch 2/1000
82/82 [==============================] - 0s 3ms/step - loss: 2.0213 - t_cpu_label_loss: 0.6342 - t_proc_label_loss: 0.4533 - t_mem_label_loss: 0.2658 - 

Test the model accuracy

In [195]:
evaluation = model.evaluate(X_test, [y_test[:, i] for i in range(5)], verbose=2)
# predictions = model.predict(X_test)
# binary_predictions = tf.round(predictions).numpy()
# accuracy = accuracy_score(y_test, binary_predictions)

# accuracy
for name, value in zip(model.metrics_names, evaluation):
    print(name, ": ", value)

26/26 - 0s - loss: 0.4080 - t_cpu_label_loss: 0.1216 - t_proc_label_loss: 0.1417 - t_mem_label_loss: 2.0676e-08 - t_store_label_loss: 0.1447 - t_script_label_loss: 3.1326e-08 - t_cpu_label_accuracy: 0.9658 - t_proc_label_accuracy: 0.9683 - t_mem_label_accuracy: 1.0000 - t_store_label_accuracy: 0.9438 - t_script_label_accuracy: 1.0000 - 41ms/epoch - 2ms/step
loss :  0.4079872667789459
t_cpu_label_loss :  0.12160632014274597
t_proc_label_loss :  0.14167839288711548
t_mem_label_loss :  2.0675939182979164e-08
t_store_label_loss :  0.1447025090456009
t_script_label_loss :  3.132583259457533e-08
t_cpu_label_accuracy :  0.9658119678497314
t_proc_label_accuracy :  0.9682539701461792
t_mem_label_accuracy :  1.0
t_store_label_accuracy :  0.9438339471817017
t_script_label_accuracy :  1.0


In [197]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming `model` is your trained model and X_test, y_test are your test data
# Also assuming you have predictions from your model
predictions = model.predict(X_test)

# Initialize an empty list to store precision, recall, and F1 scores for each class
precision_scores = []
recall_scores = []
f1_scores = []

# Iterate over each class (assuming there are 5 classes)
for i in range(5):
    true_labels = y_test[:, i]
    predicted_labels = (predictions[i] > 0.5).astype(int)  # Assuming threshold is 0.5, adjust as needed
    
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    
    print(f"Metrics for Class {i}: Precision = {precision:.4f}, Recall = {recall:.4f}, F1 Score = {f1:.4f}")

# Calculate and print the average precision, recall, and F1 scores
average_precision = sum(precision_scores) / len(precision_scores)
average_recall = sum(recall_scores) / len(recall_scores)
average_f1 = sum(f1_scores) / len(f1_scores)

print(f"\nAverage Metrics: Precision = {average_precision:.4f}, Recall = {average_recall:.4f}, F1 Score = {average_f1:.4f}")

26/26 [==============================] - 0s 1ms/step
Metrics for Class 0: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Metrics for Class 1: Precision = 0.9683, Recall = 1.0000, F1 Score = 0.9839
Metrics for Class 2: Precision = 0.0000, Recall = 0.0000, F1 Score = 0.0000
Metrics for Class 3: Precision = 0.9438, Recall = 1.0000, F1 Score = 0.9711
Metrics for Class 4: Precision = 1.0000, Recall = 1.0000, F1 Score = 1.0000

Average Metrics: Precision = 0.5824, Recall = 0.6000, F1 Score = 0.5910


c:\Users\Onome Israel Agwa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Onome Israel Agwa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Onome Israel Agwa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Save the model
model.save("trained.model", save_format="tf")

INFO:tensorflow:Assets written to: trained.model\assets


INFO:tensorflow:Assets written to: trained.model\assets


In [ ]:
num_samples = 10
num_features = 172

example_data = np.random.rand(num_samples, num_features)

example_data_reshaped = example_data.reshape((num_samples, 1, num_features))

# Make predictions
predictions = model.predict(example_data_reshaped)

predictions

1/1 [==============================] - 0s 304ms/step


[array([[0.08055726],
        [0.04190304],
        [0.13236296],
        [0.03208787],
        [0.01731396],
        [0.01410119],
        [0.03621501],
        [0.02381714],
        [0.05673011],
        [0.0229031 ]], dtype=float32),
 array([[0.9322028 ],
        [0.9509256 ],
        [0.8055772 ],
        [0.9495429 ],
        [0.9717216 ],
        [0.9734123 ],
        [0.9329925 ],
        [0.9392673 ],
        [0.94233465],
        [0.96535873]], dtype=float32),
 array([[0.03356308],
        [0.04388024],
        [0.04870339],
        [0.04388905],
        [0.03330357],
        [0.03299983],
        [0.049658  ],
        [0.02247985],
        [0.037178  ],
        [0.04154348]], dtype=float32),
 array([[0.9242589 ],
        [0.9354231 ],
        [0.7822914 ],
        [0.96469337],
        [0.97911   ],
        [0.9776328 ],
        [0.95100445],
        [0.9542286 ],
        [0.9250896 ],
        [0.97131515]], dtype=float32),
 array([[0.9595116 ],
        [0.9513712 ],
        

In [ ]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 36556), started 11 days, 9:56:01 ago. (Use '!kill 36556' to kill it.)

In [ ]:
conn.close()